In [ ]:
import org.apache.spark.sql.types.{StringType, StructType, BooleanType}

private val SCHEMA = (new StructType)
    .add("@type", StringType)
    .add("device", (new StructType)
      .add("deviceType", StringType)
    )
    .add("object", (new StructType) 
      .add("@id", StringType) 
      .add("@type", StringType) 
      .add("attachments", StringType)
      .add("body", StringType)
      .add("inReplyTo", StringType)
      .add("isFirstMessage", BooleanType)
      .add("origin", StringType)
      .add("receiverConversationId", StringType)
      .add("senderConversationId", StringType)
      .add("subject", StringType)
      .add("publisher", (new StructType)
          .add("@id", StringType)
      )
    )
    .add("provider", (new StructType)
      .add("@type", StringType)
      .add("component", StringType)
      .add("productType", StringType)
    )
    .add("tracker", (new StructType)
      .add("type", StringType)
      .add("version", StringType)
    )
    .add("account", (new StructType)
      .add("accountId", StringType)
    )
    .add("target", (new StructType)
      .add("accountId", StringType)
    )
    .add("schema", StringType)
    .add("@id", StringType)
    .add("published",StringType)

val events = spark.read.schema(SCHEMA).json("s3://schibsted-spt-common-prod/purple/messaging/client=subito/version=1/year=2017/month=10/day=24/hour=10")


events.cache
events.printSchema()

In [ ]:
import org.apache.spark.sql.functions._
import java.util.UUID

private val selectAdID = (value: String) => {
    ":ad(.*)list".r.findFirstIn(value.toLowerCase()).get.replace(":ad","").replace("list","")
}

private val reduceUUID = (value: String) => {
        UUID.nameUUIDFromBytes(value.getBytes()).toString();
}

private val checkIfAttachment = (value: String) => {
        if (value == null || value.equals("[]")){ "false"; }
        else { "true"}
}

spark.udf.register("select_ad_id", selectAdID)
spark.udf.register("reduce_UUID", reduceUUID)
spark.udf.register("check_if_attachment", checkIfAttachment)

val messages = events.
    filter(($"object.@type" === "Message")).
    select(col("@type").alias("status"),
           col("device.devicetype").alias("device"),
           col("object.@type").alias("type"),
           col("object.attachments").alias("attachments"),
           col("object.body").alias("body"),
           col("object.inReplyTo").alias("adId"),
           col("object.isFirstMessage").alias("isFirstMessage"),
           col("object.origin").alias("origin"),
           col("object.receiverConversationId").alias("receiverConversationId"),
           col("object.senderConversationId").alias("senderConversationId"),
           col("object.subject").alias("subject"),
           col("account.accountId").alias("sender_userId"),
           col("target.accountId").alias("recipient_userId"),
           col("published")).
    cache

messages.createOrReplaceTempView("messages")

In [ ]:
val data = messages.
    sqlContext.sql("SELECT status, device, check_if_attachment(attachments), select_ad_id(adId), isFirstMessage, origin, reduce_UUID(senderConversationId), reduce_UUID(receiverConversationId), sender_userId, recipient_userId, published  from messages").
    cache.show(300,false)

In [ ]:
val data = messages.
    sqlContext.sql("SELECT status, device, type, isFirstMessage, origin, category, count(*) from messages group by status, device, type, isFirstMessage, origin, category order by device, origin").
    cache


data.show(100, false)

In [ ]:
val data = messages.
    sqlContext.sql("SELECT select_ad_id(adId) , count(*) from messages group by adId order by count(*) desc").
    cache


data.show(1000, false)

In [ ]:
val data = messages.
    sqlContext.sql("SELECT * from  messages limit 1 ").
    cache


data.show(1000, false)